In [11]:
import pandas as pd

In [12]:
ls

data/           people_states_split.ipynb  tidy_data_exercise_solution.ipynb
docs/           README.md
markdown.ipynb  tidy_data_exercise.ipynb


In [13]:
# The data is in a sub-folder called "data"
# read_excel will read the first sheet in the workbook if you don't specify another

ps = pd.read_excel('./data/PeopleStates.xlsx')
ps

,name,states
0,Bobby,"Wyoming,Michigan"
1,Sue,"Wisconsin,Nevada,California"
2,Tamika,"Florida,Washington"
3,Cale,South Dakota
4,Iris,"Washington,Oregon,California"


In [14]:
# string operations will be applied to each row
# will end up with a single column of lists if don't put expand=True

psplit = ps.states.str.split(',', expand=True)
psplit

,0,1,2
0,Wyoming,Michigan,None
1,Wisconsin,Nevada,California
2,Florida,Washington,None
3,South Dakota,None,None
4,Washington,Oregon,California


In [15]:
# concat will use the index to align rows

pexp = pd.concat([ps.name, psplit], axis=1)
pexp

,name,0,1,2
0,Bobby,Wyoming,Michigan,None
1,Sue,Wisconsin,Nevada,California
2,Tamika,Florida,Washington,None
3,Cale,South Dakota,None,None
4,Iris,Washington,Oregon,California


In [16]:
# id_vars will be repeated and not un-pivoted

ptidy = pd.melt(pexp, id_vars=['name'], value_name='state')
ptidy.head(8)

,name,variable,state
0,Bobby,0,Wyoming
1,Sue,0,Wisconsin
2,Tamika,0,Florida
3,Cale,0,South Dakota
4,Iris,0,Washington
5,Bobby,1,Michigan
6,Sue,1,Nevada
7,Tamika,1,Washington


In [17]:
# since we didn't specify a var_name for melt(), it defaulted to "variable"
# can specify a list to select only certain columns, dropping others not needed

ptidy = ptidy[['name','state']]
ptidy

,name,state
0,Bobby,Wyoming
1,Sue,Wisconsin
2,Tamika,Florida
3,Cale,South Dakota
4,Iris,Washington
5,Bobby,Michigan
6,Sue,Nevada
7,Tamika,Washington
8,Cale,None
9,Iris,Oregon


In [18]:
# many methods include an "inplace" argument, so it won't make a copy
# NOTE: you're writing over your data in place!

ptidy.dropna(inplace=True)

In [19]:
ptidy.sort_values(by='name', inplace=True)
ptidy

,name,state
0,Bobby,Wyoming
5,Bobby,Michigan
3,Cale,South Dakota
4,Iris,Washington
9,Iris,Oregon
14,Iris,California
1,Sue,Wisconsin
6,Sue,Nevada
11,Sue,California
2,Tamika,Florida


In [20]:
sp = pd.read_excel('./data/PeopleStates.xlsx', sheetname='Sheet2')
sp.tail(5)

,state,population_2010,house_seats
45,South Dakota,814191,1
46,North Dakota,672591,1
47,Alaska,710249,1
48,Vermont,625745,1
49,Wyoming,563767,1


In [21]:
ppop = pd.merge(ptidy, sp, how='left', left_on='state', right_on='state')
ppop.sort_values('population_2010', ascending=False, inplace=True)
ppop

,name,state,population_2010,house_seats
5,Iris,California,37254503,53
8,Sue,California,37254503,53
9,Tamika,Florida,18804623,27
1,Bobby,Michigan,9884129,14
3,Iris,Washington,6724543,10
10,Tamika,Washington,6724543,10
6,Sue,Wisconsin,5687289,8
4,Iris,Oregon,3831073,5
7,Sue,Nevada,2700691,4
2,Cale,South Dakota,814191,1


In [25]:
# Usually we can save to an Excel file, but we'd need to install another module
# so, save as JSON for now. There are multiple "orientations"
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_json.html

ppop.to_json('./data/PeopleStates_Merged.json', orient='records')